# Chap.6 数量化１類

本章では、目的変数が量的変数、説明変数が質的変数の場合の解析方法を考える。  
結論から述べると、質的変数についてはダミー変数を設定し、それを量的変数と  
捉えることで解析が可能となる。

## 6.1の説明は割愛

## 6.2 説明変数が１個の場合の解析方法

今回の説明変数は線形代数の成績である。注意すべきは、成績が得点といった  
数値ではなく、「優、良、可」といった数値ではないもので与えられている  
ことである。  
質的変数のことをアイテムと呼び、優、良、可という分類をカテゴリーとして  
定義する。具体的な数値化の方法は教科書p.88-9を参照してほしい。  
優、良、可、それぞれに対して、0 or 1でダミー変数を振ってから、回帰式の  
モデルを作成する。このとき各カテゴリー変数について、例えば成績が優である  
とき、  
x1_(2) + x2_(3) = 0  
が成立し、これは2変数について共線性が成立するため、最小２乗法が適用  
できない（正規方程式について係数行列のインバースが計算できない）。  
従い、ダミー変数を１つ削除することで、この課題を解決する。  
例えばx1_(1)削除後の成績表は6.2.csvのようになる。この新しい成績表を  
使って回帰分析を実行する。

## まずsklearnでやってみる

In [20]:
#ライブラリをインポート
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
%matplotlib inline
import matplotlib.pyplot as plt
import math
import statsmodels.api as sm

In [2]:
npArray = np.loadtxt("6.2.csv", delimiter=",", dtype="float",skiprows=1)
x = npArray[:, 1:3]#説明変数x12,x13を要素とする２次元配列を作成
y = npArray[:, 3:4].ravel()#ターゲット変数scoreを要素とする１次元配列を作成

In [3]:
#モデルの選択
model = LinearRegression()

In [4]:
#フィッティング（テスト用のデータがある場合は「学習」）
model.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [5]:
#予測値を書き出し
y_pred=model.predict(x)
print(y_pred)

[87.5 87.5 87.5 87.5 76.  76.  76.  70.  70.  70. ]


In [6]:
#回帰係数を書き出し
print('x12の回帰係数： %.2f' % model.coef_[0])
print('x13の回帰係数： %.2f' % model.coef_[1])
print('バイアス項： %.2f' % model.intercept_)

x12の回帰係数： -11.50
x13の回帰係数： -17.50
バイアス項： 87.50


In [9]:
#決定係数を出力
R_squared = model.score(x,y)
print(R_squared)

0.6114273204903677


In [11]:
#重相関係数を出力
R = math.sqrt(R_squared)
print(R)

0.781938182013366


## 自由度調整済決定係数を求めるためにstatsmodelsを使う

In [13]:
#データフレームの作成
data = pd.read_csv("6.2.csv", sep=",")

In [14]:
data

,Unnamed: 0,x12,x13,score
0,1,0,0,96
1,2,0,0,88
2,3,0,0,77
3,4,0,0,89
4,5,1,0,80
5,6,1,0,71
6,7,1,0,77
7,8,0,1,78
8,9,0,1,70
9,10,0,1,62


In [17]:
#説明変数を取り出す
X = data.loc[:, ['x12','x13']].values

#ターゲット変数を取り出す
Y = data['score'].values

In [21]:
#モデルの作成
model2 = sm.OLS(Y, sm.add_constant(X))
results = model2.fit()

#結果を表示
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.611
Model:                            OLS   Adj. R-squared:                  0.500
Method:                 Least Squares   F-statistic:                     5.507
Date:                Sun, 23 Aug 2020   Prob (F-statistic):             0.0366
Time:                        21:27:12   Log-Likelihood:                -32.037
No. Observations:                  10   AIC:                             70.07
Df Residuals:                       7   BIC:                             70.98
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         87.5000      3.561     24.574      0.0

/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  "anyway, n=%i" % int(n))


サイキットラーンの時と同じ結果が得られている。かつadjusted R^2も得られている。  
しかし、データ数が足りないという文句が来ている。ちなみにF_0 = 5.51は後で参照する。  
なお、今回は学習用データとテスト用データを準備しているわけではないため、汎化性能の議論はしない、というかできない。

## 変数選択

誤解しないでほしいのは、カテゴリー3つはあくまで説明変数1つの中に集約されているものであり  
どのカテゴリーは入れて、どのカテゴリーは入れないとかそういう話をしているのではないということ。  
カテゴリー全部まとめて突っ込むのかどうかを議論している。繰り返すが、今の場合あくまで説明変数は  
1つなのである。こんなこと書いているのも私が混乱したせいではあるが...